# Select variables : forecasting of tropical cylone data at 1-day horizon

This notebook accompanies the following publication:
Paul Platzer, Arthur Avenas, Bertrand Chapron, Lucas Drumetz, Alexis Mouche, Léo Vinour. Distance Learning for Analog Methods. 2024. [⟨hal-04841334⟩](https://hal.science/hal-04841334)

It is used to run gradient-descent optimization algorithms for numerical experiments with IBTrACS tropical cyclone data, at 1-day forecast horizon. The aim is to select the most relevant variables for this forecast and for this particular dataset.

In [1]:
import numpy as np
# import numexpr as ne
import pandas
import matplotlib.pyplot as plt
import matplotlib
from tqdm.notebook import tqdm
from sklearn.neighbors import NearestNeighbors
import os
os.environ["OMP_NUM_THREADS"] = "16"
os.environ["OPENBLAS_NUM_THREADS"] = "16"
os.environ["MKL_NUM_THREADS"] = "16"
os.environ["NUMEXPR_NUM_THREADS"] = "16"
import sys
sys.path.append('../../functions/.')
from analogs import apply_transform, find_analogues, compute_weights, compute_diffs, compute_mae_mad, compute_error
from distance_learning import compute_gradient_MSE, compute_gradient_CRPS, compute_regularization, learn_distance
from TC_utils import M, Rmax_from_M, correct_vmx_ibt, Rmxa23

In [2]:
matplotlib.rcParams.update({'font.size': 14})

In [3]:
cols = ['#377eb8', '#ff7f00', '#4daf4a',
                  '#f781bf', '#a65628', '#984ea3',
                  '#999999', '#e41a1c', '#dede00']

In [4]:
data_folder = '../../data/tropical_cyclone/'
output_folder = '../../output/tropical_cyclone/'

# Parameters for loading IBTrACS dataset

In [5]:
files = os.listdir(data_folder)

# Input variables
var_names = ['Vmax', 'Rmax_IBT', 'R34', 'fcor', 'u_trans', 'v_trans']

# Output variable to forecast: derivative of Vmax
var_y = ['Vmax']
dydt = True
ind_var_y = []
for name_tmp in var_y:
    ind_var_y.append(np.argwhere(np.array(var_names)==name_tmp)[0][0])

# Utils to compute Rmax estimate from Avenas et al. (2023)
var_A23 = ['fcor', 'Vmax', 'R34', ]
ind_A23 = []
for name_tmp in var_A23:
    ind_A23.append(np.argwhere(np.array(var_names)==name_tmp)[0][0])

# Add names of auxilliary variables (Rmax_A23 and time-derivatives)
var_names_all = var_names.copy()
var_names_all.append('Rmax_A23')
for name in var_names_all.copy():
    var_names_all.append('d'+name+'/dt')

# Add name of time since the threshold of 18m/s is crossed for Vmax
var_names_all.append('t_18')

# Loading and preprocessing dataset

In [6]:
## Set forecast time-horizon (multiple of 3hours)

h = 8


## Load dataset

IBT = np.array(pandas.read_csv(data_folder + files[0], usecols = var_names))
IBT = np.concatenate( [ IBT , 
         Rmxa23(IBT[:,ind_A23[0]] , IBT[:,ind_A23[1]] , IBT[:,ind_A23[2]]).reshape(-1,1) ,
                      ],  axis=1)
IBT = np.concatenate( ( IBT[1:] , IBT[1:] - IBT[:-1] ) , axis=1 )
IBT = np.concatenate( [ IBT ,
           3*np.arange(len(IBT)).reshape(-1,1) ],  axis=1)
train_x = IBT[0:-h,:]
train_y = IBT[h:,ind_var_y] - IBT[0:-h,ind_var_y] 
ID = np.array([0]*len(IBT[0:-h,:]))


for i in np.arange(1, len(files)):
    IBT = np.array(pandas.read_csv(data_folder + files[i], usecols = var_names))
    IBT = np.concatenate( [ IBT , 
             Rmxa23(IBT[:,ind_A23[0]] , IBT[:,ind_A23[1]] , IBT[:,ind_A23[2]]).reshape(-1,1) ,
                          ],  axis=1)
    IBT = np.concatenate( ( IBT[1:] , IBT[1:] - IBT[:-1] ) , axis=1 )
    IBT = np.concatenate( [ IBT ,
           3*np.arange(len(IBT)).reshape(-1,1) ],  axis=1)
    train_x = np.concatenate([train_x, IBT[0:-h,:]])
    train_y = np.concatenate([train_y, IBT[h:,ind_var_y] - IBT[0:-h,ind_var_y]])
    ID = np.concatenate([ID, np.array([i]*len(IBT[0:-h,:]))])

# center and reduce
mean_IBTrACS = np.mean(train_x, axis=0)
std_IBTrACS = np.std(train_x, axis=0)
mean_y = np.mean(train_y, axis=0)
std_y = np.std(train_y, axis=0)
for j in range(train_x.shape[1]):
    train_x[:,j] = (train_x[:,j] - mean_IBTrACS[j]) / std_IBTrACS[j]
for j in range(train_y.shape[1]):
    train_y[:,j] = (train_y[:,j] - mean_y[j]) / std_y[j]    

# Optimization: loop over permutations and regularization parameter

In [10]:
np.arange(0,0.012,0.002) 

array([0.   , 0.002, 0.004, 0.006, 0.008, 0.01 ])

In [ ]:
# Set parameters for optimization
transform_diagonal = np.ones(len(var_names_all))
k = 200
# Regul = np.einsum('i,j->ij', np.arange(0,11,5) , np.array([0.0001,0.00001]) ) # amplitude of regularization term in gradient-descent optimization
# regul_type = 'elastic-net'
# Regul = np.arange(0,0.016,0.001) # amplitude of regularization term in gradient-descent optimization
# Regul = np.arange(0,0.003,0.001) # amplitude of regularization term in gradient-descent optimization
Regul = np.arange(0,0.002,0.012) # amplitude of regularization term in gradient-descent optimization
regul_type = 'l1/l2'
n_epoch = None
# n_iter = 20
n_iter = 150
batch_size = 4000 # augment ??
corr_length_train = 24
Nperm = 20
# Nperm = 1
# learning_rate_factor = 1*1e1
learning_rate_factor = 5*1e0
nn_algo = 'auto'
compute_error_train = False
n_jobs=-1

A_grad = []
E_grad_batch = []
E_grad_train = []
E_grad_test = []
random_state_number = []

for i_perm in tqdm(range(Nperm)):
    # Generate random permutation (reproducible)
    random_state_number.append(1312 + i_perm)
    rs = np.random.RandomState(random_state_number[-1])
    perm = rs.permutation(len(files))
    Itest = np.argwhere(np.isin(ID, perm[:len(files)//3]))[:,0]
    Itrain = np.argwhere(np.isin(ID, perm[len(files)//3:]))[:,0]

    # Optimize using gradient-descent    
    train_X = apply_transform(train_x, transform_diagonal, Itrain)
    
    nn = NearestNeighbors( algorithm = nn_algo , 
                              n_neighbors = k + 1 + 2*corr_length_train, n_jobs=n_jobs ) # leave-one-out procedure + anticipating time-correlated data
    nn.fit(train_X)
    
    crps_init = compute_error(train_X, train_y, Itrain, Itrain, k, nn,
                              loo=True, corr_length_train=corr_length_train, vector_out=False, error_type='CRPS')
    
    learning_rate_crps = learning_rate_factor/crps_init

    # Loop over regularization intensity
    A_grad_regul = []
    E_grad_batch_regul = []
    E_grad_train_regul = []
    E_grad_test_regul = []
    for regul_coef in tqdm(Regul):
        
        result_crps = learn_distance(train_x, train_y, transform_diagonal, Itrain, Itrain, Itest, 
                   k = k, nn_algo='auto', error_type='CRPS', n_epoch=n_epoch, n_iter=n_iter,
                    learning_rate = learning_rate_crps, regul_coef = regul_coef, regul_type = regul_type,
                   loo=True, corr_length_train=corr_length_train,
                     batch_size = batch_size, compute_error_train = compute_error_train,
                         n_jobs=n_jobs, verbose_batch = True)
    
        A_grad_regul.append( result_crps[0].copy() )
        E_grad_batch_regul.append( result_crps[1].copy() )
        E_grad_train_regul.append( result_crps[2].copy() )
        E_grad_test_regul.append( result_crps[3].copy() )

        print('')
    
    A_grad.append( np.array(A_grad_regul) )
    E_grad_batch.append( np.array(E_grad_batch_regul) )
    E_grad_train.append( np.array(E_grad_train_regul) )
    E_grad_test.append( np.array(E_grad_test_regul) )
    

A_grad = np.array(A_grad)
E_grad_batch = np.array(E_grad_batch)
E_grad_train = np.array(E_grad_train)
E_grad_test = np.array(E_grad_test)

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Starting distance-learning algorithm with the following parameters:
Error type = CRPS
Transformation type = diagonal matrix (weighting of coordinates)
Number of analogues = 200
Catalogue and target sets are equal (leave-one-out procedure & leaving time-correlated analogues below 24 time-increments).
Learning rate = 12.029215894562244
Number of Epochs = 30
Mini-batch size = 4000
Regularization = [0.0]
Errors computed = mini-batch, test.
Errors not computed = train ,


  0%|          | 0/30 [00:00<?, ?it/s]

epoch 1/30   |   iter. 1/6 : 
CRPS(mini-batch) = 4.17E-01 ;  CRPS(train) = NAN ;  CRPS(test) = 4.03E-01
epoch 1/30   |   iter. 2/6 : 
CRPS(mini-batch) = 4.13E-01 ;  CRPS(train) = NAN ;  CRPS(test) = 4.01E-01
epoch 1/30   |   iter. 3/6 : 
CRPS(mini-batch) = 4.13E-01 ;  CRPS(train) = NAN ;  CRPS(test) = 3.98E-01
epoch 1/30   |   iter. 4/6 : 
CRPS(mini-batch) = 4.10E-01 ;  CRPS(train) = NAN ;  CRPS(test) = 3.96E-01
epoch 1/30   |   iter. 5/6 : 
CRPS(mini-batch) = 4.03E-01 ;  CRPS(train) = NAN ;  CRPS(test) = 3.94E-01
epoch 1/30   |   iter. 6/6 : 
CRPS(mini-batch) = 3.93E-01 ;  CRPS(train) = NAN ;  CRPS(test) = 3.92E-01
epoch 2/30   |   iter. 1/6 : 
CRPS(mini-batch) = 4.06E-01 ;  CRPS(train) = NAN ;  CRPS(test) = 3.91E-01
epoch 2/30   |   iter. 2/6 : 
CRPS(mini-batch) = 4.01E-01 ;  CRPS(train) = NAN ;  CRPS(test) = 3.90E-01
epoch 2/30   |   iter. 3/6 : 
CRPS(mini-batch) = 3.96E-01 ;  CRPS(train) = NAN ;  CRPS(test) = 3.89E-01
epoch 2/30   |   iter. 4/6 : 
CRPS(mini-batch) = 3.99E-01 ;  CRP

...

epoch 1/31   |   iter. 1/5 : 
CRPS(mini-batch) = 4.07E-01 ;  CRPS(train) = NAN ;  CRPS(test) = 4.15E-01
epoch 1/31   |   iter. 2/5 : 
CRPS(mini-batch) = 4.13E-01 ;  CRPS(train) = NAN ;  CRPS(test) = 4.12E-01
epoch 1/31   |   iter. 3/5 : 
CRPS(mini-batch) = 3.98E-01 ;  CRPS(train) = NAN ;  CRPS(test) = 4.10E-01
epoch 1/31   |   iter. 4/5 : 
CRPS(mini-batch) = 3.94E-01 ;  CRPS(train) = NAN ;  CRPS(test) = 4.08E-01
epoch 1/31   |   iter. 5/5 : 
CRPS(mini-batch) = 4.02E-01 ;  CRPS(train) = NAN ;  CRPS(test) = 4.06E-01
epoch 2/31   |   iter. 1/5 : 
CRPS(mini-batch) = 4.04E-01 ;  CRPS(train) = NAN ;  CRPS(test) = 4.05E-01
epoch 2/31   |   iter. 2/5 : 
CRPS(mini-batch) = 3.98E-01 ;  CRPS(train) = NAN ;  CRPS(test) = 4.03E-01
epoch 2/31   |   iter. 3/5 : 
CRPS(mini-batch) = 3.97E-01 ;  CRPS(train) = NAN ;  CRPS(test) = 4.02E-01
epoch 2/31   |   iter. 4/5 : 
CRPS(mini-batch) = 3.85E-01 ;  CRPS(train) = NAN ;  CRPS(test) = 4.01E-01
epoch 2/31   |   iter. 5/5 : 
CRPS(mini-batch) = 3.88E-01 ;  CRP

  0%|          | 0/1 [00:00<?, ?it/s]

Starting distance-learning algorithm with the following parameters:
Error type = CRPS
Transformation type = diagonal matrix (weighting of coordinates)
Number of analogues = 200
Catalogue and target sets are equal (leave-one-out procedure & leaving time-correlated analogues below 24 time-increments).
Learning rate = 12.148389295680298
Number of Epochs = 30
Mini-batch size = 4000
Regularization = [0.0]
Errors computed = mini-batch, test.
Errors not computed = train ,


  0%|          | 0/30 [00:00<?, ?it/s]

epoch 1/30   |   iter. 1/6 : 
CRPS(mini-batch) = 4.11E-01 ;  CRPS(train) = NAN ;  CRPS(test) = 4.09E-01
epoch 1/30   |   iter. 2/6 : 
CRPS(mini-batch) = 4.10E-01 ;  CRPS(train) = NAN ;  CRPS(test) = 4.06E-01
epoch 1/30   |   iter. 3/6 : 
CRPS(mini-batch) = 4.01E-01 ;  CRPS(train) = NAN ;  CRPS(test) = 4.03E-01
epoch 1/30   |   iter. 4/6 : 
CRPS(mini-batch) = 4.07E-01 ;  CRPS(train) = NAN ;  CRPS(test) = 4.01E-01
epoch 1/30   |   iter. 5/6 : 
CRPS(mini-batch) = 4.05E-01 ;  CRPS(train) = NAN ;  CRPS(test) = 3.99E-01
epoch 1/30   |   iter. 6/6 : 
CRPS(mini-batch) = 4.08E-01 ;  CRPS(train) = NAN ;  CRPS(test) = 3.97E-01
epoch 2/30   |   iter. 1/6 : 
CRPS(mini-batch) = 3.99E-01 ;  CRPS(train) = NAN ;  CRPS(test) = 3.95E-01
epoch 2/30   |   iter. 2/6 : 
CRPS(mini-batch) = 3.98E-01 ;  CRPS(train) = NAN ;  CRPS(test) = 3.93E-01
epoch 2/30   |   iter. 3/6 : 
CRPS(mini-batch) = 3.99E-01 ;  CRPS(train) = NAN ;  CRPS(test) = 3.92E-01
epoch 2/30   |   iter. 4/6 : 
CRPS(mini-batch) = 3.87E-01 ;  CRP

In [8]:
for i_perm in range(Nperm):
    for j_regul in range(len(Regul)):
        plt.figure(figsize=(4,3))
        plt.title('Regul='+str(Regul[j_regul]))
        for k_coef in range(A_grad.shape[-1]):
            plt.plot(A_grad[i_perm,j_regul,:,k_coef], color=(cols*2)[k_coef])
        plt.show()

In [27]:
# # Since the shape will depend on the number and size of tropical cyclones selected, we want to put all resulting data in the same format

# # Target shape from first outcome
# target_shape_A = A_grad[0].shape
# target_shape_E_batch = E_grad_batch[0].shape
# target_shape_E_train = E_grad_train[0].shape
# target_shape_E_test = E_grad_test[0].shape

# # Function to pad arrays
# def pad_array(arr, target_shape):
#     padded = np.full(target_shape, np.nan)  # Create an array filled with NaNs
#     padded[:, :arr.shape[1]] = arr  # Copy the original data
#     return padded

# # Pad arrays where needed
# A_grad_arrays = [pad_array(arr, target_shape_A) for arr in A_grad]
# E_grad_batch_arrays = [pad_array(arr, target_shape_E_batch) for arr in E_grad_batch]
# E_grad_train_arrays = [pad_array(arr, target_shape_E_train) for arr in E_grad_train]
# E_grad_test_arrays = [pad_array(arr, target_shape_E_test) for arr in E_grad_test]

# # Convert to NumPy array
# A_grad_array = np.array(A_grad_arrays)
# E_grad_batch_array = np.array(E_grad_batch_arrays)
# E_grad_train_array = np.array(E_grad_train_arrays)
# E_grad_test_array = np.array(E_grad_test_arrays)

In [8]:
# Save
np.savez(output_folder + 'select_vars_TC_gradient_descent.npz', allow_pickle=True,
        var_y = np.array(var_y),
        var_names_all = np.array(var_names_all),
        # A_grad = A_grad_array,
        # E_grad_batch = E_grad_batch_array,
        # E_grad_train = E_grad_train_array,
        # E_grad_test = E_grad_test_array,
        A_grad = np.array(A_grad),
        E_grad_batch = np.array(E_grad_batch),
        E_grad_train = np.array(E_grad_train),
        E_grad_test = np.array(E_grad_test),
        h = h,
        k = k,
         batch_size = batch_size,
         n_epoch = n_epoch,
        corr_length_train = corr_length_train,
        learning_rate_factor = learning_rate_factor,
        Regul = Regul,
        regul_type = regul_type,
        random_state_number = random_state_number,
)